In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import pickle as pkl
import warnings
import shutil
from collections import Counter
warnings.filterwarnings("ignore")

import torch
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
from torchvision import datasets

from tqdm.notebook import tqdm

# Dataset download

In [19]:
!rm -r dataset
%mkdir dataset
!cp -v drive/MyDrive/TechSoc\ IHC\ 2021/dataset/dataset.zip dataset/dataset.zip
%cd dataset
#!gdown https://drive.google.com/uc?id=1if1LofYV6T_iKB2uphcDVnhrUGq_vgU1
#!gdown https://drive.google.com/uc?id=1WuwM5b-kG3-26EydEvwdMxeGeIBZ4Uix
!gdown https://drive.google.com/uc?id=1YV-B51V4_IsPMl9FVis4dhm_lUmsPtSX
!7z x dataset.zip
!7z x annotations.zip
!rm dataset.zip
!rm annotations.zip
%cd ..
%mkdir saved_weights
%mkdir submissions

'drive/MyDrive/TechSoc IHC 2021/dataset/dataset.zip' -> 'dataset/dataset.zip'
/content/dataset
Downloading...
From: https://drive.google.com/uc?id=1YV-B51V4_IsPMl9FVis4dhm_lUmsPtSX
To: /content/dataset/annotations.zip
100% 147k/147k [00:00<00:00, 46.4MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 15584540052 bytes (15 GiB)

Extracting archive: dataset.zip
 10% 4096 Open              --
Path = dataset.zip
Type = zip
Physical Size = 15584540052
64-bit = +

  0%      0% 48 - test/104.png                        0% 82 - test/1070.png                         0% 115 - test/110.png                       

# Dataloader

In [20]:
class IHC_train_dataset():
    def __init__(self, root: str, annFile: str, clsFile: str, transform = None):
        self.root = root
        self.annFile = annFile
        self.clsFile = clsFile
        self.transform = transform
        self.data = pkl.load(open(annFile, 'rb'))
        self.ids = list(self.data.keys())

    def __getitem__(self, index: int):
        """
        Args:
            index (int): Index

        Returns:
            tuple: Tuple (image, class_id, class_val)
        """
        img_id = self.ids[index]
        target = self.data[img_id]
        image = Image.open(os.path.join(self.root, img_id)).convert('RGB')
        
        if self.transform is not None:
            image = self.transform(image)
        
        class_id = torch.tensor([target], dtype=torch.int64)
        
        return image, class_id


    def __len__(self):
        return len(self.ids)
    
    def __get_classes__(self):
        categories = pkl.load(open(self.clsFile, 'rb')) 
        return categories

In [21]:
class train_dataloader():
    def __init__(self, dataset, batch_size : int, shuffle : bool):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        if shuffle:
            self.index_sampler = torch.randperm(len(self.dataset))
        else:
            self.index_sampler = torch.tensor(list(range(len(self.dataset))))
        self.ids = iter(self.index_sampler)
        
    def _reset(self):
        self.index_sampler = torch.randperm(len(self.dataset))
        self.ids = iter(self.index_sampler)
    
    def _next_index(self):
        return next(self.ids)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        image_list = []
        class_id_list = []
        
        if self.ids is None:
            self._reset()
        
        for i in range(self.batch_size):
            id = self._next_index()
            image, class_id = self.dataset[id]
            
            image_list.append(torch.unsqueeze(image, 0))
            class_id_list.append(class_id)
        
        batch_images = torch.cat(image_list, 0)
        batch_class_ids = torch.cat(class_id_list, 0)

        return batch_images, batch_class_ids
    
    def __len__(self):
        return len(self.index_sampler)
    
    next = __next__

In [22]:
class IHC_test_dataset():
    def __init__(self, root: str, transform = None):
        self.root = root
        self.transform = transform
        self.ids = ['{0}.png'.format(i) for i in range(5000)]

    def __getitem__(self, index: int):
        """
        Args:
            index (int): Index

        Returns:
            tuple: Tuple (image)
        """
        img_id = self.ids[index]
        image = Image.open(os.path.join(self.root, img_id)).convert('RGB')
        id = torch.tensor([int(img_id.split('.')[0])], dtype=torch.int64)
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, id


    def __len__(self):
        return len(self.ids)

In [23]:
class test_dataloader():
    def __init__(self, dataset, batch_size : int, shuffle : bool):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        if shuffle:
            self.index_sampler = torch.randperm(len(self.dataset))
        else:
            self.index_sampler = torch.tensor(list(range(len(self.dataset))))
        self.ids = iter(self.index_sampler)
        
    def _reset(self):
        self.index_sampler = torch.randperm(len(self.dataset))
        self.ids = iter(self.index_sampler)
    
    def _next_index(self):
        return next(self.ids)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        image_list = []
        image_id_list = []
        
        if self.ids is None:
            self._reset()
        
        for i in range(self.batch_size):
            id = self._next_index()
            image, image_id = self.dataset[id]
            
            image_list.append(torch.unsqueeze(image, 0))
            image_id_list.append(image_id)
        
        batch_images = torch.cat(image_list, 0)
        batch_image_ids = torch.cat(image_id_list, 0)

        return batch_images, batch_image_ids
    
    def __len__(self):
        return len(self.index_sampler)
    
    next = __next__

In [24]:
train_data = './dataset/train'
train_ann = './dataset/train_annotations.pkl'
class_file = './dataset/class_labels.pkl'

val_data ='./dataset/train'
val_ann ='./dataset/val_annotations.pkl'
class_file = './dataset/class_labels.pkl'

test_data ='./dataset/test'

train_transform = transforms.Compose([#transforms.Resize(256),
                                      #transforms.Resize(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                           std=[0.229, 0.224, 0.225]),
                                      ])
val_transform = transforms.Compose([#transforms.Resize(256),
                                    #transforms.Resize(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                         std=[0.229, 0.224, 0.225]),
                                    ])
test_transform = transforms.Compose([#transforms.Resize(256),
                                     #transforms.Resize(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                          std=[0.229, 0.224, 0.225]),
                                     ])

In [25]:
trainset = IHC_train_dataset(root=train_data, annFile=train_ann,
                             clsFile=class_file, transform=train_transform)
trainloader = train_dataloader(trainset, batch_size=50, shuffle=True)

valset = IHC_train_dataset(root=val_data, annFile=val_ann,
                           clsFile=class_file, transform=val_transform)
valloader = train_dataloader(valset, batch_size=50, shuffle=True)

testset = IHC_test_dataset(root=test_data, transform=val_transform)
testloader = test_dataloader(testset, batch_size=50, shuffle=False)

In [26]:
print('Number of training samples:', len(trainset))
print('Number of validation samples:', len(valset))
print('Number of test samples:', len(testset))
print('Categories :', trainset.__get_classes__())

Number of training samples: 28016
Number of validation samples: 6984
Number of test samples: 5000
Categories : {'real': 0, 'fake': 1}


# Model

In [27]:
import torch
import torch.nn as nn
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152', 'resnext50_32x4d', 'resnext101_32x8d',
           'wide_resnet50_2', 'wide_resnet101_2']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
    'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=2, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=11, stride=4, padding=5,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x

    def forward(self, x):
        return self._forward_impl(x)


def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)


def resnet34(pretrained=False, progress=True, **kwargs):
    r"""ResNet-34 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet34', BasicBlock, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)


def resnet50(pretrained=False, progress=True, **kwargs):
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)


def resnet101(pretrained=False, progress=True, **kwargs):
    r"""ResNet-101 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet101', Bottleneck, [3, 4, 23, 3], pretrained, progress,
                   **kwargs)


def resnet152(pretrained=False, progress=True, **kwargs):
    r"""ResNet-152 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet152', Bottleneck, [3, 8, 36, 3], pretrained, progress,
                   **kwargs)


def resnext50_32x4d(pretrained=False, progress=True, **kwargs):
    r"""ResNeXt-50 32x4d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 4
    return _resnet('resnext50_32x4d', Bottleneck, [3, 4, 6, 3],
                   pretrained, progress, **kwargs)


def resnext101_32x8d(pretrained=False, progress=True, **kwargs):
    r"""ResNeXt-101 32x8d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 8
    return _resnet('resnext101_32x8d', Bottleneck, [3, 4, 23, 3],
                   pretrained, progress, **kwargs)


def wide_resnet50_2(pretrained=False, progress=True, **kwargs):
    r"""Wide ResNet-50-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_
    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _resnet('wide_resnet50_2', Bottleneck, [3, 4, 6, 3],
                   pretrained, progress, **kwargs)


def wide_resnet101_2(pretrained=False, progress=True, **kwargs):
    r"""Wide ResNet-101-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_
    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _resnet('wide_resnet101_2', Bottleneck, [3, 4, 23, 3],
                   pretrained, progress, **kwargs)

# Train

In [28]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [29]:
def train_epoch(model, train_loader, optimizer, epoch):
    model.train()
    cls_loss = nn.NLLLoss()
    logsoftmax = nn.LogSoftmax(dim=1)
    total_cls_loss = 0.0
    correct = 0
    count = 0
    for i, data in tqdm(enumerate(train_loader)):
        images, targets = data
        images = images.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        loss = cls_loss(logsoftmax(outputs), targets)
        
        loss.backward()
        optimizer.step()
        
        total_cls_loss += loss.item()
        correct += (predicted == targets).sum().item()
        count += targets.size(0)

        print('\r', 'Epoch : {0}, Iteration : {1} ==> Training loss : {2:.3f}, Accuracy : {3:.1f}%'.
                format(epoch+1, i+1, total_cls_loss/(i+1), 100*correct/count), end='')
        
    return model, total_cls_loss/(i+1)

# Validation

In [30]:
def val_epoch(model, val_loader, epoch):
    model.eval()
    cls_loss = nn.NLLLoss()
    logsoftmax = nn.LogSoftmax(dim=1)
    total_cls_loss = 0.0
    correct = 0
    count = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(val_loader)):
            images, targets = data
            images = images.to(device)
            targets = targets.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            loss = cls_loss(logsoftmax(outputs), targets)
            
            total_cls_loss += loss.item()
            correct += (predicted == targets).sum().item()
            count += targets.size(0)

            print('\r', 'Epoch : {0}, Iteration : {1} ==> Validation loss : {2:.3f}, Accuracy : {3:.1f}%'.
                    format(epoch+1, i+1, total_cls_loss/(i+1), 100*correct/count), end='')
    
    return total_cls_loss/(i+1)

# Test

In [31]:
def test(model, test_loader, save_path):
    model.eval()
    test_ids = []
    test_p_real = []
    test_predictions = []
    softmax = torch.nn.Softmax(dim=1)
    with torch.no_grad():
        for i, data in tqdm(enumerate(test_loader)):
            images, ids = data
            images = images.to(device)
            
            outputs = model(images)
            
            outputs = softmax(outputs)
            _, predicted = torch.max(outputs.data, 1)
            predicted = predicted.cpu().numpy()
            outputs = outputs[:, 0].squeeze().cpu().numpy()
            ids = ids.squeeze().cpu().numpy()
            
            test_ids.extend(list(ids))
            test_p_real.extend(list(outputs))
            test_predictions.extend(list(predicted))

    print('No. of images tested: {0}'.format(len(test_ids)))
    print('Distribution of classes:', Counter(test_predictions))
    print(max(test_p_real), min(test_p_real))

    df = pd.DataFrame(list(zip(test_ids, test_p_real)), columns=['id', 'p_real'])
    df.to_csv(save_path, index=False)

# ResNet-18

In [32]:
device = torch.device('cuda')
print("Device :", device)

num_epochs = 30
lr = 1e-4
best_vloss = 10
best_epoch = 0

model = resnet18(pretrained=False)
model = model.to(device)

optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-3)
schdl = torch.optim.lr_scheduler.CosineAnnealingLR(optim, num_epochs, eta_min=5e-6)

Device : cuda


In [33]:
for epoch in range(num_epochs):
    path = './saved_weights/resnet-18.pt'
    
    trainloader = train_dataloader(trainset, batch_size=50, shuffle=True)
    model, tloss = train_epoch(model, trainloader, optim, epoch)
    
    valloader = train_dataloader(valset, batch_size=50, shuffle=True)
    vloss = val_epoch(model, valloader, epoch)
    
    schdl.step()
    
    if vloss<best_vloss:
        print('Saving....')
        model_dict = {'epoch':epoch+1,
                      'model_state_dict':model.state_dict(),
                      'loss':vloss}
        torch.save(model_dict, path)
        best_vloss = vloss
        best_epoch = epoch

 Epoch : 1, Iteration : 560 ==> Training loss : 0.415, Accuracy : 82.2%


 Epoch : 1, Iteration : 139 ==> Validation loss : 0.333, Accuracy : 85.7%
Saving....


 Epoch : 2, Iteration : 560 ==> Training loss : 0.312, Accuracy : 87.0%


 Epoch : 2, Iteration : 139 ==> Validation loss : 0.325, Accuracy : 84.9%
Saving....


 Epoch : 3, Iteration : 560 ==> Training loss : 0.273, Accuracy : 88.8%


...

 Epoch : 29, Iteration : 139 ==> Validation loss : 0.070, Accuracy : 97.3%


 Epoch : 30, Iteration : 560 ==> Training loss : 0.058, Accuracy : 97.8%


 Epoch : 30, Iteration : 139 ==> Validation loss : 0.141, Accuracy : 94.5%


 Epoch : 31, Iteration : 31 ==> Training loss : 0.052, Accuracy : 98.5%

KeyboardInterrupt: ignored

In [34]:
testset = IHC_test_dataset(root=test_data, transform=test_transform)
testloader = test_dataloader(testset, batch_size=50, shuffle=True)
csv_path = './submissions/sub_22_4_21_resnet-18.csv'

path = './saved_weights/resnet-18.pt'
model = resnet18(pretrained=False)
model = model.to(device)
model_dict = torch.load(path)
model.load_state_dict(model_dict['model_state_dict'])

test(model, testloader, csv_path)


No. of images tested: 5000
Distribution of classes: Counter({1: 3351, 0: 1649})
1.0 2.5126028e-06


In [37]:
dir = 'saved_weights'
for file in os.listdir(dir):
    shutil.copyfile(os.path.join(dir, file), os.path.join('drive/MyDrive/TechSoc IHC 2021/', dir, file))

dir = 'submissions'
for file in os.listdir(dir):
    shutil.copyfile(os.path.join(dir, file), os.path.join('drive/MyDrive/TechSoc IHC 2021/', dir, file))

# ResNet-34

In [38]:
device = torch.device('cuda')
print("Device :", device)

num_epochs = 30
lr = 1e-4
best_vloss = 10
best_epoch = 0

model = resnet34(pretrained=False)

model = model.to(device)

optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-3)
schdl = torch.optim.lr_scheduler.CosineAnnealingLR(optim, num_epochs, eta_min=5e-6)

Device : cuda


In [ ]:
#valloader = train_dataloader(valset, batch_size=50, shuffle=True)
#vloss = val_epoch(model, valloader, epoch=-1)
    
for epoch in range(num_epochs):
    path = './saved_weights/resnet-34.pt'
    
    trainloader = train_dataloader(trainset, batch_size=50, shuffle=True)
    model, tloss = train_epoch(model, trainloader, optim, epoch)
    
    valloader = train_dataloader(valset, batch_size=50, shuffle=True)
    vloss = val_epoch(model, valloader, epoch)
    
    schdl.step()
    
    if vloss<best_vloss:
        print('Saving....')
        model_dict = {'epoch':epoch+1,
                      'model_state_dict':model.state_dict(),
                      'loss':vloss}
        torch.save(model_dict, path)
        best_vloss = vloss
        best_epoch = epoch

 Epoch : 1, Iteration : 560 ==> Training loss : 0.414, Accuracy : 81.3%


 Epoch : 1, Iteration : 139 ==> Validation loss : 0.358, Accuracy : 83.7%
Saving....


 Epoch : 2, Iteration : 560 ==> Training loss : 0.326, Accuracy : 86.1%


 Epoch : 2, Iteration : 139 ==> Validation loss : 0.303, Accuracy : 86.6%
Saving....


 Epoch : 3, Iteration : 560 ==> Training loss : 0.288, Accuracy : 88.2%


 Epoch : 3, Iteration : 139 ==> Validation loss : 0.260, Accuracy : 88.8%
Saving....


 Epoch : 4, Iteration : 275 ==> Training loss : 0.257, Accuracy : 89.3%

In [ ]:
testset = IHC_test_dataset(root=test_data, transform=test_transform)
testloader = test_dataloader(testset, batch_size=50, shuffle=True)
csv_path = './submissions/sub_22_4_21_resnet-34.csv'

path = './saved_weights/resnet-34.pt'
model = resnet34(pretrained=False)

model = model.to(device)
model_dict = torch.load(path)
model.load_state_dict(model_dict['model_state_dict'])

test(model, testloader, csv_path)

In [ ]:
dir = 'saved_weights'
for file in os.listdir(dir):
    shutil.copyfile(os.path.join(dir, file), os.path.join('drive/MyDrive/TechSoc IHC 2021/', dir, file))

dir = 'submissions'
for file in os.listdir(dir):
    shutil.copyfile(os.path.join(dir, file), os.path.join('drive/MyDrive/TechSoc IHC 2021/', dir, file))

# ResNet-50

In [ ]:
device = torch.device('cuda')
print("Device :", device)

num_epochs = 30
lr = 1e-4
best_vloss = 10
best_epoch = 0

model = resnet50(pretrained=False)

model = model.to(device)

optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-3)
schdl = torch.optim.lr_scheduler.CosineAnnealingLR(optim, num_epochs, eta_min=5e-6)

In [ ]:
#valloader = train_dataloader(valset, batch_size=50, shuffle=True)
#vloss = val_epoch(model, valloader, epoch=-1)
    
for epoch in range(num_epochs):
    path = './saved_weights/resnet-50.pt'
    
    trainloader = train_dataloader(trainset, batch_size=50, shuffle=True)
    model, tloss = train_epoch(model, trainloader, optim, epoch)
    
    valloader = train_dataloader(valset, batch_size=50, shuffle=True)
    vloss = val_epoch(model, valloader, epoch)
    
    schdl.step()
    
    if vloss<best_vloss:
        print('Saving....')
        model_dict = {'epoch':epoch+1,
                      'model_state_dict':model.state_dict(),
                      'loss':vloss}
        torch.save(model_dict, path)
        best_vloss = vloss
        best_epoch = epoch

In [ ]:
testset = IHC_test_dataset(root=test_data, transform=test_transform)
testloader = test_dataloader(testset, batch_size=50, shuffle=True)
csv_path = './submissions/sub_22_4_21_resnet-50.csv'

path = './saved_weights/resnet-50.pt'
model = resnet50(pretrained=False)

model = model.to(device)
model_dict = torch.load(path)
model.load_state_dict(model_dict['model_state_dict'])

test(model, testloader, csv_path)

In [ ]:
dir = 'saved_weights'
for file in os.listdir(dir):
    shutil.copyfile(os.path.join(dir, file), os.path.join('drive/MyDrive/TechSoc IHC 2021/', dir, file))

dir = 'submissions'
for file in os.listdir(dir):
    shutil.copyfile(os.path.join(dir, file), os.path.join('drive/MyDrive/TechSoc IHC 2021/', dir, file))

# ResNet-101

In [ ]:
device = torch.device('cuda')
print("Device :", device)

num_epochs = 30
lr = 1e-4
best_vloss = 10
best_epoch = 0

model = resnet101(pretrained=False)

model = model.to(device)

optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-3)
schdl = torch.optim.lr_scheduler.CosineAnnealingLR(optim, num_epochs, eta_min=5e-6)

In [ ]:
#valloader = train_dataloader(valset, batch_size=50, shuffle=True)
#vloss = val_epoch(model, valloader, epoch=-1)
    
for epoch in range(num_epochs):
    path = './saved_weights/resnet-101.pt'
    
    trainloader = train_dataloader(trainset, batch_size=50, shuffle=True)
    model, tloss = train_epoch(model, trainloader, optim, epoch)
    
    valloader = train_dataloader(valset, batch_size=50, shuffle=True)
    vloss = val_epoch(model, valloader, epoch)
    
    schdl.step()
    
    if vloss<best_vloss:
        print('Saving....')
        model_dict = {'epoch':epoch+1,
                      'model_state_dict':model.state_dict(),
                      'loss':vloss}
        torch.save(model_dict, path)
        best_vloss = vloss
        best_epoch = epoch

In [ ]:
testset = IHC_test_dataset(root=test_data, transform=test_transform)
testloader = test_dataloader(testset, batch_size=50, shuffle=True)
csv_path = './submissions/sub_22_4_21_resnet-101.csv'

path = './saved_weights/resnet-101.pt'
model = resnet101(pretrained=False)

model = model.to(device)
model_dict = torch.load(path)
model.load_state_dict(model_dict['model_state_dict'])

test(model, testloader, csv_path)

In [ ]:
dir = 'saved_weights'
for file in os.listdir(dir):
    shutil.copyfile(os.path.join(dir, file), os.path.join('drive/MyDrive/TechSoc IHC 2021/', dir, file))

dir = 'submissions'
for file in os.listdir(dir):
    shutil.copyfile(os.path.join(dir, file), os.path.join('drive/MyDrive/TechSoc IHC 2021/', dir, file))

# ResNet-152

In [ ]:
device = torch.device('cuda')
print("Device :", device)

num_epochs = 30
lr = 1e-4
best_vloss = 10
best_epoch = 0

model = resnet152(pretrained=False)

model = model.to(device)

optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-3)
schdl = torch.optim.lr_scheduler.CosineAnnealingLR(optim, num_epochs, eta_min=5e-6)

In [ ]:
#valloader = train_dataloader(valset, batch_size=50, shuffle=True)
#vloss = val_epoch(model, valloader, epoch=-1)
    
for epoch in range(num_epochs):
    path = './saved_weights/resnet-152.pt'
    
    trainloader = train_dataloader(trainset, batch_size=50, shuffle=True)
    model, tloss = train_epoch(model, trainloader, optim, epoch)
    
    valloader = train_dataloader(valset, batch_size=50, shuffle=True)
    vloss = val_epoch(model, valloader, epoch)
    
    schdl.step()
    
    if vloss<best_vloss:
        print('Saving....')
        model_dict = {'epoch':epoch+1,
                      'model_state_dict':model.state_dict(),
                      'loss':vloss}
        torch.save(model_dict, path)
        best_vloss = vloss
        best_epoch = epoch

In [ ]:
testset = IHC_test_dataset(root=test_data, transform=test_transform)
testloader = test_dataloader(testset, batch_size=50, shuffle=True)
csv_path = './submissions/sub_22_4_21_resnet-152.csv'

path = './saved_weights/resnet-152.pt'
model = resnet152(pretrained=False)

model = model.to(device)
model_dict = torch.load(path)
model.load_state_dict(model_dict['model_state_dict'])

test(model, testloader, csv_path)

In [ ]:
dir = 'saved_weights'
for file in os.listdir(dir):
    shutil.copyfile(os.path.join(dir, file), os.path.join('drive/MyDrive/TechSoc IHC 2021/', dir, file))

dir = 'submissions'
for file in os.listdir(dir):
    shutil.copyfile(os.path.join(dir, file), os.path.join('drive/MyDrive/TechSoc IHC 2021/', dir, file))